# CMAPSS — RNN / LSTM / GRU con Cross-Validation

In [1]:
import sys, os, time
print('Python', sys.version)
try:
    import torch
    print('PyTorch', torch.__version__)
    print('CUDA available:', torch.cuda.is_available())
    if torch.cuda.is_available():
        print('CUDA version reported by torch:', torch.version.cuda)
        print('GPU count:', torch.cuda.device_count())
        print('GPU name:', torch.cuda.get_device_name(0))
except Exception as e:
    print('PyTorch import error:', e)

Python 3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]
PyTorch 2.5.1+cu121
CUDA available: True
CUDA version reported by torch: 12.1
GPU count: 1
GPU name: NVIDIA GeForce RTX 3050 Laptop GPU


In [ ]:
BASE_PATH = r"C:\Users\David\Documents\Master-Big-Data-Data-Sciencee-e-Inteligencia-Artificial\TFM\AeroGPT\data\CMAPSS"
RAW_PATH = os.path.join(BASE_PATH, "raw")
MODEL_PATH = os.path.join(BASE_PATH, "models_CV")
FIG_PATH = os.path.join(BASE_PATH, "figures_CV")
METRICS_PATH = os.path.join(BASE_PATH, "metrics_CV")
REPORTS_PATH = os.path.join(BASE_PATH, "reports_CV")

for p in [MODEL_PATH, FIG_PATH, METRICS_PATH, REPORTS_PATH]:
    os.makedirs(p, exist_ok=True)

In [3]:
import numpy as np, pandas as pd, joblib, matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GroupKFold
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', DEVICE)

Using device: cuda


## 3) Carga y preprocesado

Funciones para cargar los archivos CMAPSS, añadir RUL, comprobar nulos, eliminar columnas constantes, escalar y crear secuencias

In [4]:
def load_fd_dataset(fd, raw_path=RAW_PATH):
    col_names = ['unit_nr','time_cycles','setting_1','setting_2','setting_3'] + [f's_{i}' for i in range(1,22)]
    train_file = os.path.join(raw_path, f"train_{fd}.txt")
    test_file  = os.path.join(raw_path, f"test_{fd}.txt")
    rul_file   = os.path.join(raw_path, f"RUL_{fd}.txt")
    train_df = pd.read_csv(train_file, sep='\s+', header=None, names=col_names)
    test_df  = pd.read_csv(test_file,  sep='\s+', header=None, names=col_names)
    rul_df   = pd.read_csv(rul_file,  sep='\s+', header=None, names=['RUL'])
    return train_df, test_df, rul_df

def add_rul(train_df):
    max_cycles = train_df.groupby('unit_nr')['time_cycles'].max().reset_index()
    max_cycles.columns = ['unit_nr','max_cycle']
    df = train_df.merge(max_cycles, on='unit_nr', how='left')
    df['RUL'] = df['max_cycle'] - df['time_cycles']
    return df.drop(columns=['max_cycle'])

def check_and_report_raw(train_df, test_df, rul_df, fd, reports_path=REPORTS_PATH):
    lines = []
    lines.append('=== Nulos train ===')
    lines.append(str(train_df.isnull().sum()[train_df.isnull().sum()>0]))
    lines.append('=== Nulos test ===')
    lines.append(str(test_df.isnull().sum()[test_df.isnull().sum()>0]))
    n_units_test = test_df['unit_nr'].nunique()
    lines.append(f'Unidades test: {n_units_test}, RUL rows: {len(rul_df)}')
    lines.append(f'Filas train: {len(train_df)}, filas test: {len(test_df)}')
    report_file = os.path.join(reports_path, f'raw_check_{fd}.txt')
    with open(report_file,'w') as f:
        f.write('\n'.join(lines))
    print('\n'.join(lines))
    return report_file

def remove_constant_columns(train_df, test_df, feature_cols):
    dropped = []
    for c in feature_cols:
        combined = pd.concat([train_df[c], test_df[c]])
        if combined.nunique() <= 1 or np.isclose(combined.std(), 0.0):
            dropped.append(c)
    if dropped:
        print('Dropped const cols:', dropped)
        train_df = train_df.drop(columns=dropped)
        test_df = test_df.drop(columns=dropped)
    else:
        print('No constant cols found.')
    return train_df, test_df, dropped

def scale_features(train_df, test_df, feature_cols):
    train_df[feature_cols] = train_df[feature_cols].fillna(train_df[feature_cols].mean())
    test_df[feature_cols]  = test_df[feature_cols].fillna(train_df[feature_cols].mean())
    scaler = StandardScaler()
    train_df[feature_cols] = scaler.fit_transform(train_df[feature_cols])
    test_df[feature_cols]  = scaler.transform(test_df[feature_cols])
    return train_df, test_df, scaler

def create_sequences_with_units(df, feature_cols, target_col, window_size=30):
    X,y,units = [],[],[]
    for unit in sorted(df['unit_nr'].unique()):
        udf = df[df['unit_nr']==unit].sort_values('time_cycles')
        feats = udf[feature_cols].values
        tgts  = udf[target_col].values
        L = len(udf)
        if L >= window_size:
            for i in range(L-window_size+1):
                seq = feats[i:i+window_size]
                if np.isnan(seq).any(): continue
                X.append(seq); y.append(tgts[i+window_size-1]); units.append(unit)
    return np.array(X), np.array(y), np.array(units)

def create_last_window_test(test_df, rul_df, feature_cols, window_size=30):
    X_test, y_test, units = [], [], []
    for i, unit in enumerate(sorted(test_df['unit_nr'].unique())):
        udf = test_df[test_df['unit_nr']==unit].sort_values('time_cycles')
        seq = udf[feature_cols].values
        if len(seq) < window_size:
            pad = np.zeros((window_size - len(seq), seq.shape[1]))
            seq = np.vstack([pad, seq])
        else:
            seq = seq[-window_size:]
        X_test.append(seq); y_test.append(rul_df.iloc[i,0]); units.append(unit)
    return np.array(X_test), np.array(y_test), np.array(units)


## 4) Modelos: GRU, LSTM y RNN

In [5]:
class BaseRNN(nn.Module):
    def __init__(self, rnn_layer, input_dim, hidden_dims=(256,128), dropout=0.3, num_layers=1):
        super().__init__()
        # usamos num_layers=1 para cada bloque y dropout solo si num_layers>1
        self.rnn1 = rnn_layer(input_dim, hidden_dims[0], batch_first=True, dropout=0.0, num_layers=num_layers)
        self.rnn2 = rnn_layer(hidden_dims[0], hidden_dims[1], batch_first=True, dropout=0.0, num_layers=num_layers)
        self.linear = nn.Linear(hidden_dims[1], 1)

    def forward(self, x):
        out,_ = self.rnn1(x)
        out,_ = self.rnn2(out)
        out = out[:, -1, :]
        return self.linear(out)


class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1=256, hidden_dim2=128, dropout=0.3):
        super().__init__()
        self.gru1 = nn.GRU(input_dim, hidden_dim1, num_layers=2, dropout=dropout, batch_first=True)
        self.gru2 = nn.GRU(hidden_dim1, hidden_dim2, num_layers=2, dropout=dropout, batch_first=True)
        self.linear = nn.Linear(hidden_dim2, 1)

    def forward(self, x):
        out, _ = self.gru1(x)
        out, _ = self.gru2(out)
        out = out[:, -1, :]
        return self.linear(out)



class LSTMModel(BaseRNN):
    def __init__(self, input_dim):
        super().__init__(nn.LSTM, input_dim, hidden_dims=(256,128), dropout=0.3)

class RNNModel(BaseRNN):
    def __init__(self, input_dim):
        super().__init__(nn.RNN, input_dim, hidden_dims=(256,128), dropout=0.3)


## 5) Funciones entrenamiento y evaluación

Funciones para entrenar una época, evaluar y entrenar con early stopping y ReduceLROnPlateau.

In [6]:
def train_one_epoch(model, optimizer, criterion, dataloader):
    model.train()
    total_loss = 0.0
    for bx, by in dataloader:
        bx, by = bx.to(DEVICE), by.to(DEVICE)
        optimizer.zero_grad()
        out = model(bx)
        loss = criterion(out, by)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * bx.size(0)
    return total_loss / len(dataloader.dataset)

def eval_model(model, criterion, dataloader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for bx, by in dataloader:
            bx, by = bx.to(DEVICE), by.to(DEVICE)
            out = model(bx)
            preds.append(out.cpu().numpy())
            trues.append(by.cpu().numpy())
    if not preds:
        return np.inf, np.inf, np.inf
    preds = np.vstack(preds); trues = np.vstack(trues)
    mse = mean_squared_error(trues, preds)
    mae = mean_absolute_error(trues, preds)
    rmse = np.sqrt(mse)
    return mse, mae, rmse

def fit_model_with_earlystop(model, train_loader, val_loader, n_epochs=200, lr=5e-4, weight_decay=1e-5, patience=20):
    model = model.to(DEVICE)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=10, factor=0.5)
    best_val = np.inf; best_state = None; history = []; wait = 0
    for epoch in range(1, n_epochs+1):
        train_loss = train_one_epoch(model, optimizer, criterion, train_loader)
        val_mse, val_mae, val_rmse = eval_model(model, criterion, val_loader)
        history.append([epoch, train_loss, val_mse, val_mae, val_rmse])
        scheduler.step(val_mse)
        if val_mse < best_val:
            best_val = val_mse
            best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}
            wait = 0
        else:
            wait += 1
        if epoch%10==0 or epoch==1:
            print(f'Epoch {epoch} | train {train_loss:.4f} | val_mse {val_mse:.4f} | val_mae {val_mae:.4f}')
        if wait >= patience:
            print('Early stopping, best val_mse=', best_val)
            break
    if best_state is not None:
        model.load_state_dict({k:v.to(DEVICE) for k,v in best_state.items()})
    return model, pd.DataFrame(history, columns=['epoch','train_loss','val_mse','val_mae','val_rmse'])


## 6) Preprocesado, CV por grupos, entrenamiento y guardado

In [7]:
FD_LIST = ['FD001','FD002','FD003','FD004']
WINDOW_SIZE = 50
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

for fd in FD_LIST:
    print('\n=====', fd, '=====')
    train_df, test_df, rul_df = load_fd_dataset(fd)
    train_df = add_rul(train_df)
    report_file = check_and_report_raw(train_df, test_df, rul_df, fd)
    FEATURE_COLS = ['setting_1','setting_2','setting_3'] + [f's_{i}' for i in range(1,22)]
    train_df, test_df, dropped = remove_constant_columns(train_df, test_df, FEATURE_COLS)
    FEATURE_COLS = [c for c in FEATURE_COLS if c not in dropped]
    train_df, test_df, scaler = scale_features(train_df, test_df, FEATURE_COLS)
    joblib.dump(scaler, os.path.join(MODEL_PATH, f'scaler_{fd}.pkl'))
    X, y, units = create_sequences_with_units(train_df, FEATURE_COLS, 'RUL', WINDOW_SIZE)
    print('Sequences:', len(X))
    if len(X)==0:
        print('No sequences for', fd); continue
    gkf = GroupKFold(n_splits=5)
    algorithms = {'GRU': GRUModel, 'LSTM': LSTMModel, 'RNN': RNNModel}
    cv_results = {alg:[] for alg in algorithms}
    for alg_name, alg_class in algorithms.items():
        print('\n--- CV', alg_name, '---')
        fold=0
        for tr_idx, va_idx in gkf.split(X, y, groups=units):
            fold += 1
            print('Fold', fold)
            Xtr = torch.tensor(X[tr_idx], dtype=torch.float32); ytr = torch.tensor(y[tr_idx], dtype=torch.float32).view(-1,1)
            Xva = torch.tensor(X[va_idx], dtype=torch.float32); yva = torch.tensor(y[va_idx], dtype=torch.float32).view(-1,1)
            tr_loader = DataLoader(TensorDataset(Xtr,ytr), batch_size=32, shuffle=True)
            va_loader = DataLoader(TensorDataset(Xva,yva), batch_size=64, shuffle=False)
            model = alg_class(input_dim=len(FEATURE_COLS))
            model, hist = fit_model_with_earlystop(model, tr_loader, va_loader, n_epochs=200, lr=5e-4, weight_decay=1e-5, patience=25)
            mse, mae, rmse = eval_model(model, nn.MSELoss(), va_loader)
            hist_file = os.path.join(METRICS_PATH, f'{fd}_{alg_name}_fold{fold}_hist.csv')
            hist.to_csv(hist_file, index=False)
            cv_results[alg_name].append({'fold':fold,'mse':mse,'mae':mae,'rmse':rmse,'hist':hist_file})
        pd.DataFrame(cv_results[alg_name]).to_csv(os.path.join(METRICS_PATH, f'{fd}_{alg_name}_cv_summary.csv'), index=False)
    # Entrenamiento final con todos los datos
    X_all = torch.tensor(X, dtype=torch.float32); y_all = torch.tensor(y, dtype=torch.float32).view(-1,1)
    full_loader = DataLoader(TensorDataset(X_all,y_all), batch_size=32, shuffle=True)
    X_test, y_test, test_units = create_last_window_test(test_df, rul_df, FEATURE_COLS, WINDOW_SIZE)
    X_test_t = torch.tensor(X_test, dtype=torch.float32); y_test_t = torch.tensor(y_test, dtype=torch.float32).view(-1,1)
    test_loader = DataLoader(TensorDataset(X_test_t,y_test_t), batch_size=64, shuffle=False)
    final_metrics = []
    for alg_name, alg_class in algorithms.items():
        print('\nFINAL TRAIN', alg_name)
        model = alg_class(input_dim=len(FEATURE_COLS))
        model, hist = fit_model_with_earlystop(model, full_loader, full_loader, n_epochs=200, lr=5e-4, weight_decay=1e-5, patience=30)
        mse, mae, rmse = eval_model(model, nn.MSELoss(), test_loader)
        print('TEST', alg_name, 'mse', mse, 'mae', mae, 'rmse', rmse)
        model_file = os.path.join(MODEL_PATH, f'{fd}_{alg_name}_final.pth')
        torch.save(model.state_dict(), model_file)
        preds = model(X_test_t.to(DEVICE)).cpu().detach().numpy().flatten()
        pd.DataFrame({'unit':test_units,'RUL_true':y_test,'RUL_pred':preds}).to_csv(os.path.join(METRICS_PATH, f'{fd}_{alg_name}_preds.csv'), index=False)
        plt.figure(figsize=(10,4)); plt.plot(y_test, label='true'); plt.plot(preds, label='pred'); plt.legend(); plt.title(f'{fd} {alg_name} test'); plt.savefig(os.path.join(FIG_PATH, f'{fd}_{alg_name}_test.png')); plt.close()
        final_metrics.append({'alg':alg_name,'mse':mse,'mae':mae,'rmse':rmse,'model_file':model_file})
    pd.DataFrame(final_metrics).to_csv(os.path.join(METRICS_PATH, f'{fd}_final_comparison.csv'), index=False)
print('Finalizado todo el procesamiento y entrenamiento.')


===== FD001 =====
=== Nulos train ===
Series([], dtype: int64)
=== Nulos test ===
Series([], dtype: int64)
Unidades test: 100, RUL rows: 100
Filas train: 20631, filas test: 13096
Dropped const cols: ['setting_3', 's_1', 's_5', 's_10', 's_16', 's_18', 's_19']
Sequences: 15731

--- CV GRU ---
Fold 1
Epoch 1 | train 7667.3068 | val_mse 6677.7910 | val_mae 61.6084
Epoch 10 | train 3215.7516 | val_mse 3590.5166 | val_mae 47.4265
Epoch 20 | train 198.3703 | val_mse 1220.5138 | val_mae 21.7096
Epoch 30 | train 22.0121 | val_mse 1344.8929 | val_mae 22.4476
Epoch 40 | train 7.4457 | val_mse 1370.7122 | val_mae 22.6359
Early stopping, best val_mse= 988.5764
Fold 2
Epoch 1 | train 7833.0091 | val_mse 6421.8447 | val_mae 61.2197
Epoch 10 | train 3275.8841 | val_mse 3350.5437 | val_mae 46.5198
Epoch 20 | train 521.9882 | val_mse 756.5701 | val_mae 16.9415
Epoch 30 | train 189.8829 | val_mse 781.6598 | val_mae 16.8592
Epoch 40 | train 40.4509 | val_mse 880.2838 | val_mae 17.6964
Early stopping, bes